In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [13]:
data = pd.read_csv("ortopedik_hastaların_biyomekanik_özellikleri_20220209.csv")
data.head(3)

,pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.027817,22.552586,39.609117,40.475232,98.672917,-0.254400,Abnormal
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,Abnormal
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,Abnormal


In [14]:
data["class"] = [1 if each == "abnormal" else 0 for each in data["class"]]
data.head()

,pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.027817,22.552586,39.609117,40.475232,98.672917,-0.254400,0
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,0
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,0
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,0
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,0


In [37]:
y = data["class"].values
x_data = data.drop(["class"],axis=1)

In [38]:
x = (x_data - np.min(x_data))/(np.max(x_data)-np.min(x_data))

In [39]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.15, random_state=1)

In [41]:
#hata veriyor, çözumu bulamadım
#destek vektör makinesi algortimasının egitim ve testi
from sklearn.svm import SVC
#destek vektör makinesinin egitimi
svm = SVC(random_state = 1)
svm.fit(x_train, y_train)

#test
print("destek vektör makinesi modeli test dogruluk: {}".format(svm.score(x_test,y_test)))


ValueError: The number of classes has to be greater than one; got 1 class

# karar agaci
* her dahili dugumun bir öznitelik uzerindeki bir testi, her dalın testin sonucu ve her yaprak dugumunun bir sınıf etiketini temsil ettigi akis semasina benzer bir yapidir.